In [1]:
import pandas as pd
import re
import numpy as np
import word2vec
import os

In [2]:
from gensim.models import word2vec

In [3]:
data = pd.read_csv('labeledTrainData.tsv',sep='\t')
del data['id']

In [4]:
#Check the current directory to see if there is mymodel
if not os.path.exists('mymodel'):
    if not os.path.exists('imdb_text'):
        data_un = pd.read_csv('unlabeledTrainData.tsv',header=0, delimiter="\t",quoting=3 )
        pat = re.compile(r'[A-Za-z]+')
        with open('imdb_text','a',encoding = 'utf-8') as f:
            for rev in data_un.review:
                str_list = pat.findall(rev)
#                str_list = [x.lower() for x in str_list]
                string = ' '.join(str_list)
                f.write(string + '\n')
            del data_un
    sentences =word2vec.Text8Corpus("imdb_text")  # load corpus  
    model =word2vec.Word2Vec(sentences, size=50)  #train skip-gram model，default window=5 
    model.save('mymodel') 
else:
    model = word2vec.Word2Vec.load('mymodel')

In [5]:
word_vectors = model.wv
del model

In [6]:
data['vec'] = data.review.apply(lambda x :[word_vectors[w] for w in x.split() if w in word_vectors])

In [7]:
import gc
gc.collect()

22

In [8]:
data = data[data['vec'].apply(lambda x:len(x)>0)]

In [9]:
data.sentiment.value_counts()

1    12500
0    12500
Name: sentiment, dtype: int64

In [10]:
maxlength = max([len(x) for x in data.vec])
maxlength

2053

In [11]:
sum(data.vec.apply(len)>300)

3940

In [12]:
def pad(x):
    if len(x)>300:
        x1 = x[:300]
    else:
        x1 = np.zeros((300,50))
        x1[:len(x)] = x
    return x1

In [13]:
data['vec'] = data.vec.apply(pad)

In [14]:
np.shape(data.sentiment.values)

(25000,)

In [20]:
data.shape

(25000, 3)

In [21]:
data.head()

,sentiment,review,vec
0,1,With all this stuff going down at the moment w...,"[[-0.88810503, -0.7569469, 2.5152059, 1.543908..."
1,1,"\The Classic War of the Worlds\"" by Timothy Hi...","[[0.5737860798835754, -1.7005704641342163, 0.4..."
2,0,The film starts with a manager (Nicholas Bell)...,"[[-2.0837579, -0.6778111, 0.713643, 2.6085117,..."
3,0,It must be assumed that those who praised this...,"[[-1.3532884120941162, 2.109499931335449, -1.4..."
4,1,Superbly trashy and wondrously unpretentious 8...,"[[0.0069475863, -0.032301027, 0.24389511, -0.0..."


In [24]:
data.sentiment.values

array([1, 1, 0, ..., 0, 0, 1])

In [26]:
data.sentiment.shape

(25000,)

In [32]:
len(data.vec[1])

300